In [8]:
import requests
from pathlib import Path
import polars as pl

SOURCE_PAGE = "https://www.insee.fr/fr/information/2120875"
SOURCE_FILE = "https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls"

BASE_DIR = Path("./data/raw/temps/nafs")
XLS_PATH = BASE_DIR / "int_courts_naf_rev_2.xls"

BASE_DIR.mkdir(parents=True, exist_ok=True)

print("📥 Téléchargement de la NAF rév. 2 (INSEE)")
print(f"🔗 Source officielle : {SOURCE_PAGE}")

r = requests.get(SOURCE_FILE)
r.raise_for_status()

with open(XLS_PATH, "wb") as f:
    f.write(r.content)

print(f"✅ Fichier téléchargé : {XLS_PATH}")


📥 Téléchargement de la NAF rév. 2 (INSEE)
🔗 Source officielle : https://www.insee.fr/fr/information/2120875
✅ Fichier téléchargé : data\raw\temps\nafs\int_courts_naf_rev_2.xls


In [10]:
import polars as pl

# LECTURE DES CODES NAFS
df = pl.read_excel(XLS_PATH)

import re

def is_naf(code: str) -> bool:
    if not isinstance(code, str):
        return False

    code = code.strip().upper()

    # Format 01.21Z  → deux chiffres, point, deux chiffres, lettre
    pattern_with_dot = r"^\d{2}\.\d{2}[A-Z]$"

    # Format 4721Z → quatre chiffres, lettre
    pattern_no_dot = r"^\d{4}[A-Z]$"

    return bool(re.match(pattern_with_dot, code) or re.match(pattern_no_dot, code))


print(df.columns) # ['ligne', 'Code', ' Intitulés de la  NAF rév. 2, version finale ', 'Intitulés NAF rév. 2, \nen 65 caractères', 'Intitulés NAF rév. 2, \nen 40 caractères']

naf_expr = pl.col("code_naf").cast(str).str.to_uppercase().str.contains(
    r"^\d{2}(\.\d{2}|\d{2})[A-Z]$"
)

df_select = df.select(
    pl.col("Code").alias("code_naf"),
    pl.lit("").alias("code_rome"),
    pl.col(" Intitulés de la  NAF rév. 2, version finale ").alias("name"),
    pl.lit("").alias("desc")
    # pl.col("Intitulés NAF rév. 2, \nen 65 caractères").alias("desc")
).filter(naf_expr)

print("Nombre de code ", df_select.count())
print(df_select.sample(20))

df_select.write_csv("naf_codes_001.csv")

['ligne', 'Code', ' Intitulés de la  NAF rév. 2, version finale ', 'Intitulés NAF rév. 2, \nen 65 caractères', 'Intitulés NAF rév. 2, \nen 40 caractères']
Nombre de code  shape: (1, 4)
┌──────────┬───────────┬──────┬──────┐
│ code_naf ┆ code_rome ┆ name ┆ desc │
│ ---      ┆ ---       ┆ ---  ┆ ---  │
│ u32      ┆ u32       ┆ u32  ┆ u32  │
╞══════════╪═══════════╪══════╪══════╡
│ 732      ┆ 732       ┆ 732  ┆ 732  │
└──────────┴───────────┴──────┴──────┘
shape: (20, 4)
┌──────────┬───────────┬─────────────────────────────────┬──────┐
│ code_naf ┆ code_rome ┆ name                            ┆ desc │
│ ---      ┆ ---       ┆ ---                             ┆ ---  │
│ str      ┆ str       ┆ str                             ┆ str  │
╞══════════╪═══════════╪═════════════════════════════════╪══════╡
│ 43.99C   ┆           ┆ Travaux de maçonnerie générale… ┆      │
│ 20.41Z   ┆           ┆ Fabrication de savons, déterge… ┆      │
│ 59.12Z   ┆           ┆ Post-production de films ciném… ┆      │

In [11]:
# Test de code NAFS
print(is_naf("01.21Z"))   # True
print(is_naf("62.01Z"))   # True
print(is_naf("4721Z"))    # False (manque le point)
print(is_naf("AB.CDZ"))   # False
print(is_naf("12.3AZ"))   # False
print(is_naf("12.34z"))   # False (lettre en minuscule)

True
True
True
False
False
True
